## AI-powered Code Autocompletion


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
!pip install transformers
!pip install datasets
from datasets import load_dataset
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import (
    get_peft_config,
    PeftModel,
    PeftConfig,
    get_peft_model,
    LoraConfig,
    TaskType,
)
import random

In [8]:
train_ds = load_dataset("ieadoboe/python-function-examples", split="train")
val_ds = load_dataset("ieadoboe/python-function-examples", split="validation")
test_ds = load_dataset("ieadoboe/python-function-examples", split="test")

In [12]:
train_ds[0]

{'repo': 'ageitgey/face_recognition',
 'path': 'examples/face_recognition_knn.py',
 'func_name': 'train',
 'original_string': 'def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo=\'ball_tree\', verbose=False):\n    """\n    Trains a k-nearest neighbors classifier for face recognition.\n\n    :param train_dir: directory that contains a sub-directory for each known person, with its name.\n\n     (View in source code to see train_dir example tree structure)\n\n     Structure:\n        <train_dir>/\n        ├── <person1>/\n        │   ├── <somename1>.jpeg\n        │   ├── <somename2>.jpeg\n        │   ├── ...\n        ├── <person2>/\n        │   ├── <somename1>.jpeg\n        │   └── <somename2>.jpeg\n        └── ...\n\n    :param model_save_path: (optional) path to save model on disk\n    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified\n    :param knn_algo: (optional) underlying data structure to support

In [13]:
def print_code_examples(dataset, num_examples=5):
    count = 0
    for example in dataset:
        print(f"\n--- Example {count+1} ---")
        print(f"Function name: {example.get('func_name', 'N/A')}")
        print(f"Docstring: {example.get('docstring', 'N/A')[:100]}...")  # Print first 100 chars
        print(f"Code:\n{example.get('code', 'N/A')}")
        count += 1
        if count >= num_examples:
            break


In [16]:
# Print training set examples
print_code_examples(train_ds, 3)


--- Example 1 ---
Function name: train
Docstring: Trains a k-nearest neighbors classifier for face recognition.

    :param train_dir: directory that ...
Code:
def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    """
    Trains a k-nearest neighbors classifier for face recognition.

    :param train_dir: directory that contains a sub-directory for each known person, with its name.

     (View in source code to see train_dir example tree structure)

     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...

    :param model_save_path: (optional) path to save model on disk
    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified
    :param knn_algo: (optional) underlying data structure

In [17]:
print_code_examples(val_ds, 3)


--- Example 1 ---
Function name: learn
Docstring: Train a deepq model.

    Parameters
    -------
    env: gym.Env
        environment to train on
  ...
Code:
def learn(env,
          network,
          seed=None,
          lr=5e-4,
          total_timesteps=100000,
          buffer_size=50000,
          exploration_fraction=0.1,
          exploration_final_eps=0.02,
          train_freq=1,
          batch_size=32,
          print_freq=100,
          checkpoint_freq=10000,
          checkpoint_path=None,
          learning_starts=1000,
          gamma=1.0,
          target_network_update_freq=500,
          prioritized_replay=False,
          prioritized_replay_alpha=0.6,
          prioritized_replay_beta0=0.4,
          prioritized_replay_beta_iters=None,
          prioritized_replay_eps=1e-6,
          param_noise=False,
          callback=None,
          load_path=None,
          **network_kwargs
            ):
    """Train a deepq model.

    Parameters
    -------
    env: gym.E

In [59]:
def collect_training_data_from_dataset(
    dataset, max_examples=1000
):
    examples = []
    for example in dataset:
        code = example.get("code", "")
        examples.append(
            {
                "function_name": example.get("func_name", ""),
                "docstring": example.get("docstring", ""),
                "code": code,
                "language": example.get("language", "python"),
            }
        )

        if len(examples) >= max_examples:
            break

    print(f"Collected {len(examples)} examples")
    return examples


# Collect training data from the dataset
training_data = collect_training_data_from_dataset(train_ds, max_examples=1000)

Collected 1000 examples


In [21]:
training_data[1]

{'function_name': 'face_distance',
 'docstring': "Given a list of face encodings, compare them to a known face encoding and get a euclidean distance\n    for each comparison face. The distance tells you how similar the faces are.\n\n    :param faces: List of face encodings to compare\n    :param face_to_compare: A face encoding to compare against\n    :return: A numpy ndarray with the distance for each face in the same order as the 'faces' array",
 'code': 'def face_distance(face_encodings, face_to_compare):\n    """\n    Given a list of face encodings, compare them to a known face encoding and get a euclidean distance\n    for each comparison face. The distance tells you how similar the faces are.\n\n    :param faces: List of face encodings to compare\n    :param face_to_compare: A face encoding to compare against\n    :return: A numpy ndarray with the distance for each face in the same order as the \'faces\' array\n    """\n    if len(face_encodings) == 0:\n        return np.empty((0

In [60]:
# Load a code-optimized tokenizer
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")


def prepare_training_examples(examples, tokenizer, max_length=512):
    """Convert raw code examples into training samples for code completion."""
    training_samples = []

    for example in examples:
        code = example["code"]
        # Skip empty or very short functions
        if len(code.strip()) < 20:
            continue

        # Tokenize the code
        tokenized = tokenizer(code, truncation=True, max_length=max_length)
        input_ids = tokenized["input_ids"]

        # Create training examples with different completion points
        # Randomly mask 10-50% of the tokens at the end
        seq_length = len(input_ids)
        if seq_length > 20:
            for _ in range(3):  # Create a few different masking positions for each example
                # Decide how much to keep (50-90% of tokens)
                keep_percent = random.uniform(0.5, 0.9)
                keep_tokens = int(seq_length * keep_percent)

                # Create input/target pairs
                input_sample = input_ids[:keep_tokens]
                target_sample = input_ids[keep_tokens:]

                training_samples.append({
                    "input_ids": input_sample,
                    "labels": target_sample,
                })

    print(f"Created {len(training_samples)} training samples from {len(examples)} examples")
    return training_samples


# Prepare the training data
training_samples = prepare_training_examples(training_data, tokenizer)

Created 3000 training samples from 1000 examples


In [48]:
training_samples[0]

{'input_ids': [4299,
  4512,
  7,
  27432,
  62,
  15908,
  11,
  2746,
  62,
  21928,
  62,
  6978,
  28,
  14202,
  11,
  299,
  62,
  710,
  394,
  32289,
  28,
  14202,
  11,
  638,
  77,
  62,
  282,
  2188,
  11639,
  1894,
  62,
  21048,
  3256,
  15942,
  577,
  28,
  25101,
  2599,
  198,
  50284,
  37811,
  198,
  50284,
  2898,
  1299,
  257,
  479,
  12,
  710,
  12423,
  12020,
  1398,
  7483,
  329,
  1986,
  9465,
  13,
  628,
  50284,
  25,
  17143,
  4512,
  62,
  15908,
  25,
  8619,
  326,
  4909,
  257,
  850,
  12,
  34945,
  329,
  1123,
  1900,
  1048,
  11,
  351,
  663,
  1438,
  13,
  628,
  50283,
  7,
  7680,
  287,
  2723,
  2438,
  284,
  766,
  4512,
  62,
  15908,
  1672,
  5509,
  4645,
  8,
  628,
  50283,
  1273,
  5620,
  25,
  198,
  50280,
  27,
  27432,
  62,
  15908,
  29,
  14,
  198,
  50280,
  6552,
  250,
  8418,
  1279,
  6259,
  16,
  29,
  14,
  198,
  50280,
  6552,
  224,
  50285,
  6552,
  250,
  8418,
  1279,
  82,
  3674,
  480,
  16,

In [26]:
# Load a pre-trained code model
model_name = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(model_name)

# Configure LoRA adapter
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# Create PEFT model
model = get_peft_model(model, peft_config)
print(
    f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}"
)
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(
    f"Percentage of trainable parameters: {100 * sum(p.numel() for p in model.parameters() if p.requires_grad) / sum(p.numel() for p in model.parameters()):.2f}%"
)

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Trainable parameters: 655360
Total parameters: 357367808
Percentage of trainable parameters: 0.18%


In [27]:
text = "def read_json(filename, print=False):"
input_ids = tokenizer(text, return_tensors="pt").input_ids

generated_ids = model.generate(input_ids, max_length=128)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


def read_json(filename, print=False):
    """
    Reads a JSON file and returns a list of dictionaries.
    """
    with open(filename, 'r') as f:
        return json.load(f)

def write_json(filename, data):
    """
    Writes a JSON file.
    """
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

def read_csv(filename, print=False):
    """
    Reads a CSV


In [61]:
class CodeCompletionDataset(Dataset):
    def __init__(self, samples, tokenizer, max_length=512):
        self.samples = samples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # Get input and target
        input_ids = sample["input_ids"]
        labels = sample["labels"]

        # Combine input with labels for training
        combined_ids = input_ids + labels

        # Handle truncation if needed
        if len(combined_ids) > self.max_length:
            combined_ids = combined_ids[:self.max_length]

        # Create attention mask
        attention_mask = [1] * len(combined_ids)

        # Pad sequences if needed
        padding_length = self.max_length - len(combined_ids)
        if padding_length > 0:
            combined_ids = combined_ids + [self.tokenizer.pad_token_id] * padding_length
            attention_mask = attention_mask + [0] * padding_length

        # Set up labels (set to -100 for input portion to ignore in loss)
        labels = [-100] * len(input_ids) + combined_ids[len(input_ids):]

        # Ensure all sequences have the right length
        if len(labels) > self.max_length:
            labels = labels[:self.max_length]
        elif len(labels) < self.max_length:
            labels = labels + [-100] * (self.max_length - len(labels))

        return {
            "input_ids": torch.tensor(combined_ids),
            "attention_mask": torch.tensor(attention_mask),
            "labels": torch.tensor(labels),
        }

def train_code_completion_model(
    training_samples,
    model_name="Salesforce/codegen-350M-mono",
    output_dir="./code-completion-model",
    num_epochs=3,
    batch_size=4,
    grad_accum_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    max_length=512,
):
    """Train a code completion model using PyTorch and Hugging Face Transformers."""
    # Load tokenizer and model
    print(f"Loading model and tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Ensure padding token is set
    if tokenizer.pad_token_id is None:
        if tokenizer.eos_token_id is not None:
            tokenizer.pad_token_id = tokenizer.eos_token_id
            print(f"Setting pad_token_id to eos_token_id: {tokenizer.pad_token_id}")
        else:
            tokenizer.pad_token_id = 0
            print("Setting pad_token_id to 0")

    # Split data into train and validation
    train_size = int(0.9 * len(training_samples))
    train_samples = training_samples[:train_size]
    val_samples = training_samples[train_size:]

    print(f"Training samples: {len(train_samples)}")
    print(f"Validation samples: {len(val_samples)}")

    # Create datasets
    train_dataset = CodeCompletionDataset(train_samples, tokenizer, max_length)
    val_dataset = CodeCompletionDataset(val_samples, tokenizer, max_length)

    # Set up training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum_steps,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir=f"{output_dir}/logs",
        logging_steps=10,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        fp16=True,  # Enable mixed-precision training
        load_best_model_at_end=True,
    )

    # Create trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    # Train the model
    print("Starting training...")
    trainer.train()

    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    return model, tokenizer

In [63]:
if __name__ == "__main__":
    # Dummy training data
    training_data = [
        {"code": "def factorial(n):\n    if n <= 1:\n        return 1\n    return n * factorial(n-1)", "function_name": "factorial"},
        {"code": "def fibonacci(n):\n    if n <= 1:\n        return n\n    return fibonacci(n-1) + fibonacci(n-2)", "function_name": "fibonacci"},
        # Add more examples...
    ]

    # Prepare the training samples
    training_samples = prepare_training_examples(training_data, tokenizer)

    # Train the model
    model, tokenizer = train_code_completion_model(training_samples)

Created 6 training samples from 2 examples
Loading model and tokenizer: Salesforce/codegen-350M-mono


Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Setting pad_token_id to eos_token_id: 50256
Training samples: 5
Validation samples: 1


OutOfMemoryError: CUDA out of memory. Tried to allocate 200.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 2376 has 14.73 GiB memory in use. Of the allocated memory 6.57 GiB is allocated by PyTorch, and 28.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [62]:

# Load a code-optimized tokenizer
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")

# Prepare training data
# (Your prepare_training_examples function as provided earlier)
training_samples = prepare_training_examples(training_data, tokenizer)

# Train the model
model, tokenizer = train_code_completion_model(
    training_samples,
    model_name="Salesforce/codegen-350M-mono",
    output_dir="./codegen-completion-model",
    num_epochs=3,
    batch_size=4
)


Created 3000 training samples from 1000 examples
Loading model and tokenizer: Salesforce/codegen-350M-mono


Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Setting pad_token_id to eos_token_id: 50256
Training samples: 2700
Validation samples: 300


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 200.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 2376 has 14.73 GiB memory in use. Of the allocated memory 6.57 GiB is allocated by PyTorch, and 28.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def generate_completion(model, tokenizer, function_prefix, max_new_tokens=100):
    inputs = tokenizer(function_prefix, return_tensors="pt")

    # Generate completion
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.95,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decode the generated tokens
    completed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Return only the newly generated part
    return completed_code[
        len(tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)) :
    ]


# Test with some examples
test_prefixes = [
    "def train_model(X_train, y_train):\n    # Create TensorFlow model\n    model = tf.keras",
    "def process_image(image_path):\n    # Load and preprocess image\n    import numpy as np\n    img = ",
    "def create_bert_classifier():\n    # Initialize a BERT model from HuggingFace\n    from transformers import ",
]

for prefix in test_prefixes:
    completion = generate_completion(model, tokenizer, prefix)
    print(f"\nPrefix:\n{prefix}")
    print(f"\nCompletion:\n{completion}")
    print("-" * 50)

In [ ]:
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments


class CodeCompletionDataset(Dataset):
    def __init__(self, samples, tokenizer, max_length=512):
        self.samples = samples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # Get input and target
        input_ids = sample["input_ids"]
        labels = sample["labels"]

        # Combine input with labels for training
        combined_ids = input_ids + labels

        # Handle truncation if needed
        if len(combined_ids) > self.max_length:
            combined_ids = combined_ids[:self.max_length]

        # Create attention mask
        attention_mask = [1] * len(combined_ids)

        # Pad sequences if needed
        padding_length = self.max_length - len(combined_ids)
        if padding_length > 0:
            combined_ids = combined_ids + [self.tokenizer.pad_token_id] * padding_length
            attention_mask = attention_mask + [0] * padding_length

        # Set up labels (set to -100 for input portion to ignore in loss)
        labels = [-100] * len(input_ids) + combined_ids[len(input_ids):]

        # Ensure all sequences have the right length
        if len(labels) > self.max_length:
            labels = labels[:self.max_length]
        elif len(labels) < self.max_length:
            labels = labels + [-100] * (self.max_length - len(labels))

        return {
            "input_ids": torch.tensor(combined_ids),
            "attention_mask": torch.tensor(attention_mask),
            "labels": torch.tensor(labels),
        }

def train_code_completion_model(
    training_samples,
    model_name="Salesforce/codegen-350M-mono",
    output_dir="./code-completion-model",
    num_epochs=3,
    batch_size=4,
    grad_accum_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    max_length=512,
):
    """Train a code completion model using PyTorch and Hugging Face Transformers."""
    # Load tokenizer and model
    print(f"Loading model and tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Ensure padding token is set
    if tokenizer.pad_token_id is None:
        if tokenizer.eos_token_id is not None:
            tokenizer.pad_token_id = tokenizer.eos_token_id
            print(f"Setting pad_token_id to eos_token_id: {tokenizer.pad_token_id}")
        else:
            tokenizer.pad_token_id = 0
            print("Setting pad_token_id to 0")

    # Split data into train and validation
    train_size = int(0.9 * len(training_samples))
    train_samples = training_samples[:train_size]
    val_samples = training_samples[train_size:]

    print(f"Training samples: {len(train_samples)}")
    print(f"Validation samples: {len(val_samples)}")

    # Create datasets
    train_dataset = CodeCompletionDataset(train_samples, tokenizer, max_length)
    val_dataset = CodeCompletionDataset(val_samples, tokenizer, max_length)

    # Set up training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum_steps,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir=f"{output_dir}/logs",
        logging_steps=10,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        fp16=True,  # Enable mixed-precision training
        load_best_model_at_end=True,
    )

    # Create trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    # Train the model
    print("Starting training...")
    trainer.train()

    # Save the fine-tuned model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    return model, tokenizer

# Example usage with dummy training data for demonstration
# In a real scenario, replace with your actual code examples
if __name__ == "__main__":
    # Dummy training data
    training_data = [
        {"code": "def factorial(n):\n    if n <= 1:\n        return 1\n    return n * factorial(n-1)", "function_name": "factorial"},
        {"code": "def fibonacci(n):\n    if n <= 1:\n        return n\n    return fibonacci(n-1) + fibonacci(n-2)", "function_name": "fibonacci"},
        # Add more examples...
    ]

    # Prepare the training samples
    training_samples = prepare_training_examples(training_data, tokenizer)

    # Train the model
    model, tokenizer = train_code_completion_model(training_samples)